In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/lib/kaggle/gcp.py
/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv
/kaggle/working/__notebook_source__.ipynb


In [2]:
from pandas import read_csv
path = '/kaggle/input/store-sales-time-series-forecasting/train.csv'
dftrain = read_csv(path, header=0)
dftrain.head()

#X=dftrain.data
X=dftrain[["id", "date","store_nbr","family","onpromotion"]]
#y=dftrain.target
y = dftrain[["sales"]]

In [3]:
pathtest = '/kaggle/input/store-sales-time-series-forecasting/test.csv'
dftest = read_csv(pathtest, header=0)
dftest.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [4]:
#from pandas import to_datetime
#from fbprophet import Prophet
# prepare expected column names
#df.columns = ['id','ds','store_nbr','family', 'y','onpromotion']
#df['ds']= to_datetime(df['ds'])
# define the model
#model = Prophet()
# fit the model
#model.fit(df)

#!pip install ESRNN

from ESRNN.m4_data import prepare_m4_data
from ESRNN.utils_evaluation import evaluate_prediction_owa

from ESRNN import ESRNN
from sklearn.model_selection import train_test_split

X_train_df, y_train_df, X_test_df, y_test_df = train_test_split(X, y, test_size=0.33, random_state=42)



In [5]:
y_test_df

,sales
2483653,2576.000000
2761877,0.000000
1217988,357.354000
1231938,5.000000
528175,0.000000
...,...
2791862,83.216000
1691971,55.404003
1611776,0.000000
1963121,0.000000


In [6]:
# Instantiate model
model = ESRNN(max_epochs=25, freq_of_test=5, batch_size=4, learning_rate=1e-4,
              per_series_lr_multip=0.8, lr_scheduler_step_size=10,
              lr_decay=0.1, gradient_clipping_threshold=50,
              rnn_weight_decay=0.0, level_variability_penalty=100,
              testing_percentile=50, training_percentile=50,
              ensemble=False, max_periods=25, seasonality=[],
              input_size=4, output_size=6,
              cell_type='LSTM', state_hsize=40,
              dilations=[[1], [6]], add_nl_layer=False,
              random_seed=1, device='cpu')

print("model fit done")


model fit done


In [7]:
model

In [8]:
# Fit model
# If y_test_df is provided the model
# will evaluate predictions on
# this set every freq_test epochs
model.fit(X_train_df, y_train_df, X_test_df, y_test_df)

# Predict on test set
#y_hat_df = model.predict(X_test_df)

# Evaluate predictions
#final_owa, final_mase, final_smape = evaluate_prediction_owa(y_hat_df, y_train_df,
                                                             #X_test_df, y_test_df,
                                                             #naive2_seasonality=1)

AssertionError: 

In [ ]:
#https://www.kaggle.com/prmohanty/python-how-to-save-and-load-ml-models

Pkl_Filename = "/kaggle/working/Pickle_RL_Model.pkl"  

import pickle
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [ ]:
# Load the Model back from file
Pkl_Filename = "/kaggle/working/Pickle_RL_Model.pkl" 
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

In [ ]:
future = Pickled_LR_Model.make_future_dataframe(periods=365)
future.tail()

In [ ]:
# Python
forecast = Pickled_LR_Model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
from pandas import read_csv
pathtest = '/kaggle/input/store-sales-time-series-forecasting/test.csv'
dftest = read_csv(pathtest, header=0)
dftest.head()

In [ ]:
# Python
forecast = Pickled_LR_Model.predict(dftest)
forecast[['id','ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
output = pd.DataFrame({'id':forecast.id, 'sales':yhat})
output

In [ ]:
output.to_csv('submission.csv', index=False)